In [1]:
import string

import tensorflow as tf

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


from nltk.corpus import stopwords

import spacy


In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
df = pd.read_csv('C:/Users/lemes/OneDrive/Рабочий стол/dz/IMDB Dataset.csv')

In [4]:
lbl_encoder = LabelEncoder()
df['sentiment'] = lbl_encoder.fit_transform(df['sentiment'])
df['sentiment']

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int32

In [5]:
words_stop = stopwords.words('english')
words_stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [6]:
translator = str.maketrans('', '', string.punctuation)
def replace_stop_words(text): # у цій функції поки не робимо лдематізацію, подивимось як відпрацює без неї
    text = text.lower()
    text = text.replace('<br />', '')
    text = text.translate(translator)
    text = text.split(' ')
    exit_exemple = [word for word in text if word not in words_stop and len(word) >= 3]
    exit_exemple = exit_exemple[-30:]
    return ' '.join(exit_exemple)

In [7]:
def lema_and_stopwords(text, how):
    text = text.replace('<br />', '')
    doc = nlp(text)
    vector = []
    for sentensce in doc.sents:
        for token in sentensce:
            if not token.is_punct and token.text.lower() not in words_stop and len(token.text) > 2:
                vector.append(token.lemma_)
    if how == 'first':
        return ' '.join(vector[:30])
    elif how == 'last':
        return ' '.join(vector[-30:])
    else:
        print('Параметр "how" повинен бути тільки "first" or "last"')
        return
                   

In [8]:
lema_and_stopwords(df['review'][0], how='last')

"'ll kill order get away well mannered middle class inmate turn prison bitch due lack street skill prison experience watch may become comfortable uncomfortable view that get touch dark side"

In [9]:
# найбільший скор мені показала модель, де ми берем останні 30 слів
df['review'] = df['review'].apply(lambda x: lema_and_stopwords(x, how='last'))

In [10]:
 df['review']

0        'll kill order get away well mannered middle c...
1        thing fantasy guard rather use traditional dre...
2        decade have never impress Scarlet Johanson man...
3        parent divorce argue like real life Jake close...
4        good Mr. Mattei direction Steve Buscemi Rosari...
                               ...                        
49995    grip soul touch allow get life get involve mov...
49996    pathetically bad figure fourth rate spoof Spri...
49997    loss life besides two former student life abor...
49998    underdeveloped be like bowling pin knock one o...
49999    goofy scene Kirk Spock McCoy Yosemite would sa...
Name: review, Length: 50000, dtype: object

In [11]:
train_df, test_df = train_test_split(df, test_size=0.3, shuffle=False)

In [12]:
vectoriztion = tf.keras.layers.TextVectorization()
vectoriztion.adapt(train_df['review'])

In [13]:
train = vectoriztion(train_df['review'])

In [14]:
train

<tf.Tensor: shape=(35000, 31), dtype=int64, numpy=
array([[  68,  181,  564, ...,  379,  384,    0],
       [  35,  903, 1946, ...,   10,  100,    0],
       [1173,   74,   44, ...,    5,  169,    0],
       ...,
       [3939, 2741, 5907, ...,  334,   85,    0],
       [ 270,  838,  830, ...,   39,   12,    0],
       [ 191,  638,  156, ...,  243,  350,    0]], dtype=int64)>

In [15]:
target = tf.keras.utils.to_categorical(train_df['sentiment'], num_classes=2)
target

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [16]:
test = vectoriztion(test_df['review'])
test 

<tf.Tensor: shape=(15000, 30), dtype=int64, numpy=
array([[  409,  1837,   965, ...,    67,   229,     1],
       [   55,    20,  7301, ...,     3,   173, 37979],
       [  114,  3358,   206, ...,     1,    10,   213],
       ...,
       [ 1299,    42,  1092, ...,  4906, 16018,  4686],
       [ 5096,    69,     9, ..., 12691,  2032,   194],
       [ 1986,    28,  3398, ...,   698,   202,     2]], dtype=int64)>

In [17]:
target_test = tf.keras.utils.to_categorical(test_df['sentiment'], num_classes=2)
target_test

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [18]:
vocabulary_size = vectoriztion.vocabulary_size()
vocabulary_size

43956

In [19]:
max_features = vocabulary_size
output_dim = 1000 # каким вектором представляется наше 1 слово
num_classes = 2

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, output_dim))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, return_sequences=True, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.0001))))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.001))))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=False, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.001))))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))

model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(loss='binary_crossentropy',  # Можно использовать 'categorical_crossentropy' при более чем 2 классах
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
# Оскільки наша модель може перенавчатися, зроибмо чекпоінт
checkpoint_path = 'model/model_epoch_v6first{epoch:02d}.h5'
checkpoint  = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=False)

In [ ]:
history = model.fit(x=train, y=target, validation_data=(test, target_test), callbacks=[checkpoint],  epochs=10)

Epoch 1/10
1094/1094 [==============================] - ETA: 0s - loss: 0.5482 - accuracy: 0.7670

C:\Users\lemes\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1094/1094 [==============================] - 597s 537ms/step - loss: 0.5482 - accuracy: 0.7670 - val_loss: 0.4522 - val_accuracy: 0.7977
Epoch 2/10
1094/1094 [==============================] - 612s 560ms/step - loss: 0.3327 - accuracy: 0.8701 - val_loss: 0.4795 - val_accuracy: 0.7980
Epoch 3/10
1094/1094 [==============================] - 569s 520ms/step - loss: 0.2300 - accuracy: 0.9179 - val_loss: 0.5122 - val_accuracy: 0.7889
Epoch 4/10
1094/1094 [==============================] - 589s 539ms/step - loss: 0.1575 - accuracy: 0.9504 - val_loss: 0.7593 - val_accuracy: 0.7819
Epoch 5/10
1094/1094 [==============================] - 595s 544ms/step - loss: 0.1154 - accuracy: 0.9666 - val_loss: 0.7674 - val_accuracy: 0.7765
Epoch 6/10
1094/1094 [==============================] - 584s 534ms/step - loss: 0.0833 - accuracy: 0.9790 - val_loss: 0.8360 - val_accuracy: 0.7711
Epoch 7/10
 781/1094 [====================>.........] - ETA: 2:37 - loss: 0.0603 - accuracy: 0.9862

In [18]:
aproove_model = tf.keras.models.load_model('model/model_epoch_v502.h5')

In [19]:
aproove_model.evaluate(test, target_test)

469/469 [==============================] - 35s 67ms/step - loss: 0.4461 - accuracy: 0.8129


[0.4460711181163788, 0.8128666877746582]

Перевіримо вже на тих коментарях що в нас були

Вирішив поділитися своїми враженнями від дворічного володінням Cadillac XT5. Рік виготовлення 2017. Повний привід, сучасний атмосферний двигун V6 об’ємом 3,6л., потужність 314 к.с. з системою відключення двох циліндрів, що дозволяє їхати як швидко розгін 7,5с. до 100 км./год.так, і за потреби, економно розхід по трасі може становити 7,5л./100 км.. Був придбаний на аукціоні з незначними пошкодженнями та пробігом 12 474 миль 20 тис.км.в 2021р. За два роки проїхав 12 тис. км. Перед ним у володінні була Ауді А4 2.0 TFSI Quattro, відповідно хотів придбати Audi Q5 до 2017р. випуску, проте потрібного варіанту не знаходилося. Всі Або були надто сильно пошкоджені, або по ціннику йшли далеко поза мій бюджет і ставали нецікавими/невигідними. Вирішив подивитися в бік біль рідкісних авто американського автопрому - Cadillac, Lincoln, Jeep. В результаті став щасливим власником Cadillac XT5 3,6 AWD. Велике, комфортне, непересічне авто, яке, ІМХО, варте уваги і дуже недооцінене у нас. Розміри між Q5 та Q7, великий багажник, хороший кліренс, якісні матеріали оздоблення. Авто абсолютно "не кохає" мізки. Після ремонту замінив рідини та фільтри й після того - виключно те, що потребував регламент. Оскільки це GM вартість фільтрів невелика. Оливу використовував Liqui Moly Molygen New Generation 5W-30, заміна раз на 10 тис. Авто чудово поводить себе як на добрих дорогах так і на недуже. Комфортне в далеких подорожах. Попри те що може швидко їхати, зовсім не провокує, як Ауді. Тут історія про комфорт та зручність. Багато місця в салоні та багажнику. Кліренс достатній для адекватних зїздів з асфальту в якийсь лісок чи за потреби десь проїхати взимку. Багато хто переживатиме за розхід, зважаючи на обєм двигуна - скажу так авто споживає не набагато більше ніж Ауді А4. В залежності від стилю їзди, місто 13-16л., траса 6,5-9 л. Вважаю, що для авто такого обєму/потужності, маси та розмірів це цілком адекватно. Хочу побажати потенційним покупцям даної марки не боятися, це "американський мерседес". З запчастинами зараз проблем немає. А гарний вигляд, непересічність авто точно не залишить вас непоміченими.

Ніколи не звертайтеся до цього салону! ЛОХОТРОН! Розводять на кредит, на машину якої немає! Для того, щоб ви не пішли, утримують документи. Коли розуміють, що ти починаєш розуміти, що це розлучення, починають дуже грубо спілкуватися. Чи не ведіться! Вам розкажуть що машина десь скоро доставлять, тільки подайте на заявку на кредит, тільки ви ніколи не дізнаєтесь на яку суму вас подали.

На жаль Спортлайф клуб для тих, хто не працює та особливо це стосуватися дітей. Навіщо взагалі продавати дитячий дитячий абонемент, якщо зверху за все потрібно доплачувати.
В мене дитина 9 років відходила зі мною рік, коли ми працювали вдома, була можливість його водити, підлаштовуючи графік. При вимушеній роботі в офісі, всі заняття в робочий час. Завезти дитину на групове заняття по абонементу може або ти з наявністю або той дорослий, який має абонемент, або супровід за 80 грн(і це за 5 хв щоб провести латину через роздягальні до тренера), або платіть за платне групове(і тоді вийде тренер). Тобто альтернативи, щоб вийшов або зустрів тренер, який проводить заняття по абонементу - немає. Платіть зверху гроші. Висновок, дитина не хоче ходити. Домучаєм абонемент і більше не куплятимо.
Вимагаєте договір, коли оплачуєте абонемент.
Бо дитині нічого не можно, лише знаходитись в дитячій кімнаті та з дорослим, але ви тоді не тренуватись.

Спорт лайф-то найгірша пропозиція в сучасному світі, мало того, що сервіс гірше не придумаєш ( тренери так собі, розклад групових лише для пенсіонерів та школярів) тай ще клуб на Дарниці закрили без попередження та права на заморозку у разі як що не хочу їздити в інший клуб, і це при такій вартості абонементів!!! Це все одно що в тебе вкрали півтора місяця від вартості абонементу в розмірі щонайменше 19 тис гривень.

In [82]:
from googletrans import Translator
translator = Translator()
review = str(input('Введіть відгук для класифікацї'))
review = translator.translate(review, src="uk", dest="en").text
review = lema_and_stopwords(review, 'last')
review = vectoriztion(review)

In [83]:
review = np.expand_dims(review, axis=0)
review

array([[  275,     1,    14,    44, 14860,   301,   120,  5259]],
      dtype=int64)

In [84]:
rezult = aproove_model.predict(review)

1/1 [==============================] - 0s 94ms/step


In [85]:
if np.argmax(rezult) == 0:
    print('positiv')
else:
    print('negativ')

positiv
